In [5]:
from openpyxl import load_workbook
wb = load_workbook('exercises/data/f1-results.xlsx')


In [6]:
import os
os.system("open exercises/data/f1-results.xlsx")

0

**Zadanie 1:** Przygotuj arkusz o nazwie *Standings*, który będzie zawierał sumaryczną liczbę punktów zawodników po każdym wyścigu. 
    
Arkusz powinien zawierać tabelą jak poniżej:

|                  | Australia | China | ... |
|------------------|-----------|-------|-----|
| Lewis Hamilton   | 18        | 43    | ... |
| Sebastian Vettel | 25        | 43    | ... |
| ...              |           |       |     |


In [12]:
# Zadanie 1

# Przygotuj dane
drivers = {}
for sheet in wb:
    for row in sheet.iter_rows():
        name = row[2].value # column 3 in xlsx
        points = row[5].value # column 6 in xlsx
        drivers.setdefault(name, {})
        races = drivers[name]
        races.setdefault(sheet.title, 0)
        races[sheet.title] = points
        
#print(drivers)

# Zapisz dane do pliku xlsx
from openpyxl import Workbook


{'Sebastian Vettel VET': {'Australia': '25', 'China': '18', 'Bahrain': '25', 'Russia': '18', 'Spain': '18', 'Monaco': '25', 'Canada': '12', 'Azerbaijan': '12', 'Austria': '18', 'Great Britain': '6', 'Hungary': '25', 'Belgium': '18', 'Italy': '15', 'Singapore': '0', 'Malaysia': '12', 'Japan': '0', 'United States': '18', 'Mexico': '12', 'Brazil': '25', 'Abu Dhabi': '15'}, 'Lewis Hamilton HAM': {'Australia': '18', 'China': '25', 'Bahrain': '18', 'Russia': '12', 'Spain': '25', 'Monaco': '6', 'Canada': '25', 'Azerbaijan': '10', 'Austria': '12', 'Great Britain': '25', 'Hungary': '12', 'Belgium': '25', 'Italy': '25', 'Singapore': '25', 'Malaysia': '18', 'Japan': '25', 'United States': '25', 'Mexico': '2', 'Brazil': '12', 'Abu Dhabi': '18'}, 'Valtteri  Bottas BOT': {'Australia': '15', 'China': '8', 'Bahrain': '15', 'Russia': '25', 'Spain': '0', 'Monaco': '12', 'Canada': '18', 'Azerbaijan': '18', 'Austria': '25', 'Great Britain': '18', 'Hungary': '15', 'Belgium': '10', 'Italy': '18', 'Singapore

**Zadanie 2:** Na podstawie powyższej tabeli, w arkuszu *Standings* umieść wykres liniowy (`LineChart`), który zaprezentuje sumaryczną liczbę punktów po kolejnych wyścigach. Na osi X powinny być nazwy wyścigów, na osi Y liczba punktów, a serie reprezentują poszczególnych kierowców. 

In [ ]:
# Zadanie 2

In [2]:
# Przyklad z: https://realpython.com/openpyxl-excel-spreadsheets-python/

import random
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference

workbook = Workbook()
sheet = workbook.active

# Let's create some sample sales data
rows = [
    ["", "January", "February", "March", "April",
    "May", "June", "July", "August", "September",
     "October", "November", "December"],
    [1, ],
    [2, ],
    [3, ],
]

for row in rows:
    sheet.append(row)

for row in sheet.iter_rows(min_row=2,
                           max_row=4,
                           min_col=2,
                           max_col=13):
    for cell in row:
        cell.value = random.randrange(5, 100)

# Create a LineChart and add the main data
chart = LineChart()
data = Reference(worksheet=sheet,
                           min_row=2,
                           max_row=4,
                           min_col=1,
                           max_col=13)
chart.add_data(data, titles_from_data=True, from_rows=True)

# Add categories to the chart
cats = Reference(worksheet=sheet,
                 min_row=1,
                 max_row=1,
                 min_col=2,
                 max_col=13)
chart.set_categories(cats)

# Rename the X and Y Axis
chart.x_axis.title = "Months"
chart.y_axis.title = "Sales (per unit)"

# Apply a specific Style
chart.style = 24

# Save!
sheet.add_chart(chart, "C6")
workbook.save("./exercises/tmp/line_chart.xlsx")

In [4]:
import os
os.system("open ./exercises/tmp/line_chart.xlsx")

0